<a href="https://colab.research.google.com/github/weso500/IOT-Anomaly-Detection/blob/main/Single_Image_VAE_A%2BIGBT_I_GenLatent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtained intially from https://keras.io/examples/generative/vae/ and https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/vae.ipynb

In [1]:
pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

Use this notebook as example for obtaining images: https://colab.research.google.com/drive/1EOFDisibXb9a4A3lZ_2z4Jbhlnx0OVy6 The below takes care of obtaining images

In [4]:
latent_dim = 3

encoder_inputs = keras.Input(shape=(256, 256, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 64, 64)     │         18,496 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 262144)         │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │     33,554,560 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling (Sampling)       │ (None, 3)              │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 33,574,726 (128.08 MB)

 Trainable params: 33,574,726 (128.08 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64 * 64 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((64, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 3)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 262144)              │       1,048,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 256, 256, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 256, 256, 3)         │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,104,835 (4.21 MB)

 Trainable params: 1,104,835 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.layers import Layer
class KLLossLayer(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

        return kl_loss

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.kl_loss_layer = KLLossLayer()

    def call(self, inputs):  # Add the call method
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = self.kl_loss_layer([z_mean, z_log_var])
        return reconstructed, kl_loss, z_mean, z_log_var

    def predict(self, x):
        reconstructed, kl_loss, z_mean, z_log_var = super().predict(x)
        return reconstructed, kl_loss, z_mean, z_log_var


In [8]:
!unzip /content/Gold_Folder_Final.zip

Streaming output truncated to the last 5000 lines.
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/108Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/109Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/10Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/110Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/111Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/112Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/113Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/114Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/115Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGB

In [9]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_C+_IGBT-I',
  image_size=(256, 256),
  batch_size=32,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds = train_ds.map(lambda x: normalization_layer(x))

Found 500 files.


In [10]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_test_ds = test_ds.map(lambda x: normalization_layer(x))

Found 190 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT

In [11]:
test_anomaly_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Anomaly/Spectrograms_Anomaly_C+_IGBT-I',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_anomaly_test_ds = test_anomaly_ds.map(lambda x: normalization_layer(x))

Found 182 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT

In [12]:
#(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
#mnist_digits = np.concatenate([x_train, x_test], axis=0)
#mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(normalized_ds, epochs=15, batch_size=32)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 31s 895ms/step - kl_loss: 222627760.0000 - loss: 222716272.0000 - reconstruction_loss: 88527.2266
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step - kl_loss: 33.5159 - loss: 44820.0508 - reconstruction_loss: 44786.5352
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - kl_loss: 76.6446 - loss: 43972.4219 - reconstruction_loss: 43895.7734
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - kl_loss: 944.8951 - loss: 40950.0312 - reconstruction_loss: 40005.1367
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - kl_loss: 1677.7856 - loss: 31226.2930 - reconstruction_loss: 29548.5078
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - kl_loss: 960.9197 - loss: 24289.9121 - reconstruction_loss: 23328.9922
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - kl_loss: 631.5424 - loss: 22508.2402 - reconstruction_loss: 21876.6992
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - kl_loss: 466.2137 - loss: 21865.9121 - reconstruction_loss: 21399.6

In [13]:
train_ds_test_ = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_A+IGBT-I',
  image_size=(256, 256),
  batch_size=1,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds_test1 = train_ds_test_.map(lambda x: normalization_layer(x))

Found 500 files.


In [14]:
pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11419 sha256=90782c356a13440a14dac3def483ff2196de6b27dafc3b813226ea36a841dcdc
  Stored in directory: /root/.cache/pip/wheels/3f/af/02/9c6556ba287b62a945d737def09b8b8c35c9e1d82b9dfae84c
Successfully built sewar


In [23]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_ds_test1.take(500):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.0032406413233630137
145.77637
[[17.002287    0.5075167   0.05972158]]
[[-3.1313348  -0.21178165 -0.19533569]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.003239520084425475
145.0678
[[16.961607    0.48359996  0.04892194]]
[[-3.118581   -0.19851258 -0.21709257]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.0028718662240468703
146.05606
[[17.024794    0.15645035  0.15390335]]
[[-3.0540648   0.13576481 -0.5074013 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.0028258513533600863
147.072
[[17.083511    0.18729408  0.14658736]]
[[-3.0867014   0.11237857 -0.48881656]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.003324678648308094
145.80385
[[17.003038    0.53290087  0.03982639]]
[[-3.133547   -0.24669996 -0.16987115]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.0032155564732850826
145.79071
[[17.003298    0.50354964  0.05272233]]
[[-3.1295078  -0.21251968 -0.1995905 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
0.002751986521611996
144.95958
[[16.960306    0.19019802  0.1429790

In [24]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df = pd.DataFrame(dict)

# saving the dataframe
df.to_csv('A+IGBT-I_train.csv')

In [26]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.0033825857558983266
149.23428
[[17.187052    0.8246835   0.01754366]]
[[-3.2490294  -0.4913409   0.06864518]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.003297295251717709
149.90279
[[17.225864    0.82216936  0.01851152]]
[[-3.2568753  -0.48671487  0.06436557]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.0034477372363062504
148.96632
[[1.7172258e+01 8.1022674e-01 1.5826812e-02]]
[[-3.2514613  -0.47749153  0.05377883]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.003531978021701987
149.12523
[[1.7182224e+01 7.9805952e-01 1.6615024e-02]]
[[-3.2533302  -0.46106306  0.03993648]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
0.003204239609704614
150.76192
[[1.7270231e+01 9.0663534e-01 8.0131553e-04]]
[[-3.2380507  -0.60294324  0.16008276]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.0032144328499292478
149.22289
[[1.7186689e+01 8.2174188e-01 1.6939392e-02]]
[[-3.2417977  -0.49496737  0.06995934]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.0032663904838659487
149.75845
[[1.72165

In [27]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df2 = pd.DataFrame(dict)

# saving the dataframe
df2.to_csv('A+IGBT-I_testnorm.csv')

In [28]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_anomaly_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.057890979495852944
196.5335
[[18.762547   -2.5857704   0.53376496]]
[[-3.9295323  3.4937308 -3.6733825]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.008591667811772617
106.102196
[[14.491064   -0.0738682   0.01738583]]
[[-2.820059    0.36287484 -0.8075009 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
0.01969644647581769
80.5803
[[12.609962   -0.3317923  -0.11624254]]
[[-2.4265442   0.51443654 -0.97581166]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0014741646120586904
148.2958
[[17.162016    0.14028437  0.1278014 ]]
[[-2.880548   -0.01277891 -0.44006544]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.030494320226099516
65.440994
[[11.381597    0.22488315 -0.2930851 ]]
[[-2.0013435  -0.2794867  -0.26684505]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.019153835948146913
93.19464
[[13.439805   -0.9981912   0.07484566]]
[[-2.8487604  1.4650463 -1.8289925]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.0014969121713295334
147.80147
[[17.132555    0.01845558  0.16460715]]
[[-2.8

In [29]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df3 = pd.DataFrame(dict)

# saving the dataframe
df3.to_csv('A+IGBT-I_testanom.csv')